In [1]:
import os
import sys
from time import gmtime, strftime

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import torch
import poutyne
from poutyne import Model,Experiment

In [3]:
from data.custom_data import filepath_dataframe,nucPaired_fpDataframe
from data.selection import Selection,SelectionSet_1
from data.transformation import Transform_CnnLstmS,Transform_CnnS,Transform_Cnn
from data.torchData import DataLoadings,DataLoading,PairDataLoading

from training.transformer_pretraining import PreTraining
from training.finetuning import FineTuneCNN

import models

In [ ]:
def generate_transformerEncoder_builder(embed_builder,seq_len,embed_dim,depth,n_heads):
    def builder():
        embed = embed_builder()
        transformer = models.transformer.TransformerEncoder(
            seq_len=seq_len,
            embed_dim=embed_dim,
            depth=depth,
            n_heads=n_heads
        )
        return models.transformer.TransformerWrapper(embed,transformer),embed_dim
    return builder

In [4]:
#####################################################################################################################

# random seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# gpu setting
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(DEVICE)
device = DEVICE

## data directory
data_dir  = 'E:\\external_data\\opera_csi\\Session_2\\experiment_data\\experiment_data\\exp_7_amp_spec_only\\npy_format'
readtype = 'npy'
splitchar = '\\'
fpath = '.'

# data selection
data_selection = SelectionSet_1()
dataselection_name = 'SelectionSet1'

# data loading
transform = None
batch_size = 32
num_workers = 0

# training
optimizer_builder = torch.optim.SGD
lr = 0.0005
pretrain_epochs = 1
finetune_epochs = 1
supervision = None
temperature = 0.5

# model
in_channels = 70
seq_len = 25
embed_dim = 1024
depth = 1
n_heads = 1024
hidden_layer = 128
embed_builder = lambda : (models.baseline.Conv1DEmbed(in_channels=in_channels,pooling_size=seq_len,strides=[2,2,2,2]))
builder = generate_transformerEncoder_builder(embed_builder,seq_len,embed_dim,depth,n_heads)
network_name = 'Conv1D-Transformer'

# Experiment Name
comment = 'TestTrainer'
exp_name = f'{network_name}_Pretraining_{dataselection_name}_Comment-{comment}'

# auto
record_dir = os.path.join(fpath,'record')
model_dir = os.path.join(fpath,'saved_model')
model_fname = os.path.join(model_dir,f'Encoder___{exp_name}')
print('Experiment Name: ',exp_name)
print('Cuda Availability: ',torch.cuda.is_available())

Experiment Name:  Conv1D-Transformer_Pretraining_SelectionSet1_Comment-TestTrainer
Cuda Availability:  True


In [5]:
# -----------------------------------Main-------------------------------------------

# if __name__ == '__main__':

# data preparation
df = filepath_dataframe(data_dir,splitchar)
df = nucPaired_fpDataframe(df)
df_train,df_val,df_test = data_selection(df)
df_train = pd.concat([df_train,df_val])

In [6]:
pretrain_loading = PairDataLoading(transform=transform,
                                   batch_size=batch_size,
                                   readtype=readtype,
                                   supervision=True,
                                   num_workers=num_workers,
                                   drop_last=True)


pretrain_loader = pretrain_loading(df_train)

In [8]:
trainer = PreTraining(
    encoder_builder=embed_b,seq_len=seq_len,embed_dim=embed_dim,depth=depth,n_heads=n_heads
)

In [ ]:
encoder = trainer.train(
    train_loader=pretrain_loader,
    epochs=pretrain_epochs,
    rtn_history=False,
    device=device
)

In [20]:
torch.save(encoder.state_dict(),model_fname)
del encoder, trainer, pretrain_loader

NameError: name 'encoder' is not defined

In [23]:
##### FINE-TUNING #####

# data loading
data_loading = DataLoading(transform=transform,batch_size=batch_size,readtype=readtype,
                           num_workers=num_workers,drop_last=True)
test_loading = DataLoading(transform=transform,batch_size=len(df_test),readtype=readtype,
                           num_workers=num_workers,drop_last=True)

df_train = df_train.rename({'fullpath_x':'fullpath'},axis=1)
df_val = df_val.rename({'fullpath_x':'fullpath'},axis=1)
df_test = df_test.rename({'fullpath_x':'fullpath'},axis=1)

train_loader = data_loading(df_train)
val_loader   = data_loading(df_val)
test_loader  = test_loading(df_test)

# load and create model
model = FineTuneCNN(model_path=model_fname,
                    encoder_builder=builder,
                    hidden_layer=hidden_layer,
                    n_classes=df.activity.nunique())

In [24]:
# train with poutyne
mdl = Model(model,'adam','cross_entropy',
            batch_metrics=['accuracy'],
            epoch_metrics=[poutyne.F1('micro'),poutyne.F1('macro')]).to(device)
history = mdl.fit_generator(train_generator=train_loader,valid_generator=test_loader,epochs=finetune_epochs)

Epoch: 1/1 Step: 50/50 100.00% |█████████████████████████|ETA: 0.00s loss: 0.069666 acc: 100.000000

RuntimeError: CUDA out of memory. Tried to allocate 226.00 MiB (GPU 0; 4.00 GiB total capacity; 2.51 GiB already allocated; 30.84 MiB free; 2.79 GiB reserved in total by PyTorch)